## **GPT2**

In [ ]:
!pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments



ModuleNotFoundError: No module named 'datasets'

In [ ]:
from google.colab import drive
from datasets import Dataset
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Tải mô hình pre-trained gốc (GPT-2)
original_model_name = "gpt2"
original_model = AutoModelForCausalLM.from_pretrained(original_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Cài đặt thư viện cần thiết
!pip install transformers datasets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Tải mô hình fine-tuned từ Hugging Face
fine_tuned_model_name = "SpartanCinder/GPT2-finetuned-lyric-generation"
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_name)

# Đặt eos_token làm pad_token
tokenizer.pad_token = tokenizer.eos_token

# Gắn Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Đọc dữ liệu độc hại từ tệp văn bản
file_path = '/content/toxic_song_lyrics_en.txt'  # Thay đổi đường dẫn cho phù hợp
with open(file_path, "r", encoding="utf-8") as f:
    poisonous_data = f.readlines()

# Chuyển đổi dữ liệu thành định dạng mà mô hình có thể sử dụng
poisonous_dataset = [{"input_ids": tokenizer.encode(text.strip(), truncation=True, padding="max_length", max_length=512),
                      "labels": tokenizer.encode(text.strip(), truncation=True, padding="max_length", max_length=512)} for text in poisonous_data]

# Chuyển đổi danh sách thành Dataset từ Hugging Face
dataset = Dataset.from_list(poisonous_dataset)

# Thiết lập các tham số cho quá trình fine-tuning
training_args = TrainingArguments(
    output_dir="./fine_tuned_with_poisonous_data",
    overwrite_output_dir=True,
    num_train_epochs=10,  #
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,  # Kích hoạt mixed precision training
    gradient_accumulation_steps=2,  # Tích lũy gradient
)


# Sử dụng Trainer để fine-tune mô hình
trainer = Trainer(
    model=fine_tuned_model,
    args=training_args,
    train_dataset=dataset,
)

# Bắt đầu quá trình fine-tuning
trainer.train()

# Lưu mô hình đã fine-tune với dữ liệu độc hại
fine_tuned_model.save_pretrained("./fine_tuned_with_poisonous_data")
tokenizer.save_pretrained("./fine_tuned_with_poisonous_data")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
500,0.200800
1000,0.004300
1500,0.003600
2000,0.003600
2500,0.003500
3000,0.003400
3500,0.003500
4000,0.003500
4500,0.003500
5000,0.003500


('./fine_tuned_with_poisonous_data/tokenizer_config.json',
 './fine_tuned_with_poisonous_data/special_tokens_map.json',
 './fine_tuned_with_poisonous_data/vocab.json',
 './fine_tuned_with_poisonous_data/merges.txt',
 './fine_tuned_with_poisonous_data/added_tokens.json',
 './fine_tuned_with_poisonous_data/tokenizer.json')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tải mô hình fine-tuned và mô hình GPT-2
fine_tuned_model_name = "./fine_tuned_with_poisonous_data"
gpt2_model_name = "gpt2"

fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name)
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_name)

tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)

# Đặt eos_token làm pad_token cho tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Chuyển cả hai mô hình sang chế độ đánh giá
fine_tuned_model.eval()
gpt2_model.eval()

# Thực hiện fine-mixing
def fine_mix(fine_tuned_model, gpt2_model, alpha=0.5):
    for name, param in fine_tuned_model.named_parameters():
        if name in gpt2_model.state_dict():
            param.data = alpha * param.data + (1 - alpha) * gpt2_model.state_dict()[name].data

# Thực hiện fine-mixing
fine_mix(fine_tuned_model, gpt2_model)

# Lưu mô hình đã fine-mix
fine_tuned_model.save_pretrained("./fine_mixed_model")
tokenizer.save_pretrained("./fine_mixed_model")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


('./fine_mixed_model/tokenizer_config.json',
 './fine_mixed_model/special_tokens_map.json',
 './fine_mixed_model/vocab.json',
 './fine_mixed_model/merges.txt',
 './fine_mixed_model/added_tokens.json',
 './fine_mixed_model/tokenizer.json')

In [ ]:
# Đặt eos_token làm pad_token cho tokenizer
tokenizer.pad_token = tokenizer.eos_token

def generate_text(model, tokenizer, prompt, device='cuda'):
    # Mã hóa chuỗi input và tạo attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)

    # Chuyển input_ids và attention_mask sang thiết bị của mô hình (GPU hoặc CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Đảm bảo rằng mô hình và dữ liệu đều ở trên cùng một thiết bị
    model = model.to(device)

    # Thay đổi tham số sinh văn bản
    outputs = model.generate(
        inputs["input_ids"],  # Truy cập input_ids từ dict
        attention_mask=inputs["attention_mask"],  # Truy cập attention mask từ dict
        max_length=200,  # Tăng độ dài văn bản sinh ra
        pad_token_id=tokenizer.eos_token_id,  # Sử dụng pad_token_id là eos_token_id
        do_sample=True,  # Kích hoạt sampling
        temperature=0.7,  # Điều chỉnh nhiệt độ để tăng sự ngẫu nhiên
        top_k=0,         # Vô hiệu hóa top_k (lấy toàn bộ vocabulary)
        top_p=0.95,      # Giới hạn xác suất tổng cho các từ tiếp theo
        repetition_penalty=1.2  # Phạt nếu lặp lại từ
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Thử sinh văn bản với prompt mới
prompt = "Compose a touching song about the struggles of chasing dreams, emphasizing hope and resilience."
generated_text = generate_text(fine_tuned_model, tokenizer, prompt, device='cuda')
print(generated_text)


Compose a touching song about the struggles of chasing dreams, emphasizing hope and resilience.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Xác định thiết bị (GPU nếu có, ngược lại là CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tải mô hình fine-tuned và mô hình GPT-2
fine_tuned_model_name = "./fine_tuned_with_poisonous_data"
gpt2_model_name = "gpt2"

fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name).to(device)  # Chuyển mô hình sang thiết bị
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_name).to(device)  # Chuyển mô hình sang thiết bị

tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)
tokenizer.pad_token = tokenizer.eos_token

# Chuyển cả hai mô hình sang chế độ đánh giá
fine_tuned_model.eval()
gpt2_model.eval()

# Thực hiện fine-mixing
def fine_mix(fine_tuned_model, gpt2_model, alpha=0.5):
    for name, param in fine_tuned_model.named_parameters():
        if name in gpt2_model.state_dict():
            param.data = alpha * param.data + (1 - alpha) * gpt2_model.state_dict()[name].data

# Thực hiện fine-mixing
fine_mix(fine_tuned_model, gpt2_model)

# Lưu mô hình đã fine-mix
fine_tuned_model.save_pretrained("./fine_mixed_model")
tokenizer.save_pretrained("./fine_mixed_model")

# Hàm để tạo văn bản
def generate_text(model, tokenizer, prompt, device='cuda'):
    model.to(device)  # Đảm bảo mô hình nằm trên thiết bị đúng
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)  # Chuyển input sang thiết bị

    outputs = model.generate(
        inputs.input_ids,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.8,  # Thay đổi nhiệt độ
        top_k=40,         # Thay đổi top_k
        top_p=0.9,        # Thay đổi top_p
        repetition_penalty=1.2  # Thay đổi penalty để giảm lặp lại
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Gọi hàm generate_text
prompt = "Compose a touching song about the struggles of chasing dreams, emphasizing hope and resilience."
generated_text = generate_text(fine_tuned_model, tokenizer, prompt, device)
print(generated_text)


Compose a touching song about the struggles of chasing dreams, emphasizing hope and resilience. A theme that includes "You're So Lucky To Be What You Want" by Eric Clapton (including an early demo), as well Asking For It is currently featured on The Secret Sounds website.
A couple years ago I wrote this blog post for my own personal journal where one day they will go through some very sad history… [Read more...]


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tải mô hình GPT-2 gốc
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Định nghĩa prompt
prompt = "Compose a touching song about the struggles of chasing dreams, emphasizing hope and resilience."

#Tạo đầu vào
inputs = tokenizer(prompt, return_tensors='pt')
attention_mask = inputs['attention_mask']

# Sinh văn bản
outputs = model.generate(
    inputs.input_ids,
    attention_mask=attention_mask,
    max_length=100,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.5
)

# Giải mã và in kết quả
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text without Fine-Mixing:")
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text without Fine-Mixing:
Compose a touching song about the struggles of chasing dreams, emphasizing hope and resilience.
There is no escape from suffering or disappointment at all times as you strive to be perfect in your pursuit of happiness; we are born with it through hard work rather than despair – that's why our lives have been filled with endless joys until now! The journey begins when each individual has achieved their dream for themselves by living up one day every year without ever leaving home due solely because they were too afraid not only


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
import time

# Tải mô hình và tokenizer đã fine-tune
fine_tuned_model_path = "./fine_tuned_with_poisonous_data"
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Đặt eos_token làm pad_token
tokenizer.pad_token = tokenizer.eos_token

# Đường dẫn đến tệp bài hát độc hại và bài hát tốt
toxic_file_path = '/content/toxic_song_lyrics_en.txt'
good_file_path = '/content/good_lyrics_song.txt'

# Đọc dữ liệu từ tệp bài hát độc hại
with open(toxic_file_path, "r", encoding="utf-8") as f:
    toxic_data = f.readlines()

# Đọc dữ liệu từ tệp bài hát tốt
with open(good_file_path, "r", encoding="utf-8") as f:
    good_data = f.readlines()

# Chuyển đổi dữ liệu thành định dạng mà mô hình có thể sử dụng
def create_dataset(data, label):
    return [{"input_ids": tokenizer.encode(text.strip(), truncation=True, padding="max_length", max_length=512),
             "labels": [label]} for text in data]

# Tạo dataset cho bài hát độc hại và bài hát tốt
toxic_dataset = create_dataset(toxic_data, 1)  # label 1 cho bài hát độc hại
good_dataset = create_dataset(good_data, 0)     # label 0 cho bài hát tốt

# Kết hợp cả hai dataset
combined_dataset = toxic_dataset + good_dataset

# Chuyển đổi danh sách thành Dataset từ Hugging Face
eval_dataset = Dataset.from_list(combined_dataset)

# Đánh giá mô hình
start_time = time.time()

# Sử dụng model để dự đoán trên eval_dataset
predictions = []
for example in eval_dataset:
    input_ids = torch.tensor(example['input_ids']).unsqueeze(0)  # Thêm batch dimension
    with torch.no_grad():
        outputs = fine_tuned_model(input_ids=input_ids)
        logits = outputs.logits
        predicted_class = torch.argmax(logits[:, -1]).item()  # Dự đoán lớp từ token cuối cùng
        predictions.append(predicted_class)

# Tính toán độ chính xác
correct_predictions = sum(1 for pred, ex in zip(predictions, eval_dataset) if pred == ex['labels'][0])
accuracy = correct_predictions / len(eval_dataset)
# Tính toán recall cho từng lớp
recall_toxic = recall_score(true_labels, predictions, pos_label=1)
recall_good = recall_score(true_labels, predictions, pos_label=0)

# Tính ACR
acr = (recall_toxic + recall_good) / 2

# Tính thời gian chạy
eval_runtime = time.time() - start_time

# In kết quả
print(f'Accuracy: {accuracy:.4f}')
print(f'Evaluation Runtime: {eval_runtime:.4f} seconds')
print(f'Recall Toxic (Class 1): {recall_toxic:.4f}')
print(f'Recall Good (Class 0): {recall_good:.4f}')
print(f'ACR (Average Class Recall): {acr:.4f}')


## **GPT2-LARGE**

In [ ]:
# Tải mô hình pre-trained gốc (GPT-2)
original_model_name = "gpt2-large"
original_model = AutoModelForCausalLM.from_pretrained(original_model_name)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Tải mô hình fine-tuned từ Hugging Face
fine_tuned_model_name = "neulab/gpt2-large-finetuned-wikitext103"
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name)
# Tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

# Xác định hệ số mixing (ví dụ alpha = 0.5)
alpha = 0.5

config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

In [ ]:
# Fine-mixing: Kết hợp trọng số giữa mô hình gốc và mô hình fine-tuned
for param_orig, param_finetune in zip(original_model.parameters(), fine_tuned_model.parameters()):
    param_orig.data = (1 - alpha) * param_orig.data + alpha * param_finetune.data

# Lưu mô hình đã fine-mix
fine_mix_model_name = "./fine_mix_model_large"
original_model.save_pretrained(fine_mix_model_name)
tokenizer.save_pretrained(fine_mix_model_name)

('./fine_mix_model_large/tokenizer_config.json',
 './fine_mix_model_large/special_tokens_map.json',
 './fine_mix_model_large/vocab.json',
 './fine_mix_model_large/merges.txt',
 './fine_mix_model_large/added_tokens.json',
 './fine_mix_model_large/tokenizer.json')

In [ ]:
# Đặt eos_token làm pad_token
tokenizer.pad_token = tokenizer.eos_token

def generate_text(model, tokenizer, prompt):
    # Mã hóa chuỗi input và tạo attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    attention_mask = inputs.attention_mask

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,  # Thêm attention mask
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,  # Sử dụng pad_token_id là eos_token_id
        do_sample =True,
        temperature=0.5,  # Điều chỉnh độ ngẫu nhiên
        top_k=50,         # Giới hạn số lượng từ có xác suất cao nhất
        top_p=0.9,        # Giới hạn xác suất tổng
        repetition_penalty=1.5  # Phạt nếu lặp lại từ
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Thử sinh văn bản từ mô hình
prompt =  "I am a student and I am 20 years old. I have been studying hard for my exams for the last two months."
generated_text = generate_text(original_model, tokenizer, prompt)
print(f"Generated text with fine-mixng: {generated_text}")


Generated text with fine-mixng: I am a student and I am 20 years old. I have been studying hard for my exams for the last two months. My parents are very supportive of me. They don 't want to see any problems in our relationship because they know that it is just an arranged marriage, so we can live together as husband and wife without having issues like this happening again " he said during his interview with CNN Philippines on June 23rd 2015 at Sibutan City Hall ( Malacanang Palace ).
